# Using SimpleNamespace with JSON

In one of my recent videos I presented the `benedict` library. One of the facets of that library was the ability to access dictionary items using attribute access instead of the standard `["..."]["..."]` style.

Some people complained about the lack of purity, and, correctly, pointed out that although Raymond Hettinger had added this to core Python, it was later retracted.

I'm not here to debate whether this was correct and will leave these debates to the experts.

However, Raymond's point was a pragmatic one, and a good one.

We often have to deal with ad-hoc JSON data and we want to easily access data at various locations in its, usually, nested structure. 

Sure, we can do this:

In [1]:
import json

json_data= '''{
    "circle": {
        "center": {
            "x": 0,
            "y": 0
        },
        "radius": 1
    },
    "rectangle": {
        "top_left": {
            "x": -50,
            "y": 50
        },
        "bottom_right": {
            "x": 50,
            "y": -50
        }
    }
}'''

In [2]:
data = json.loads(json_data)

And then we can access the data thusly:

In [3]:
center = data["circle"]["center"]["x"], data["circle"]["center"]["y"]
center

(0, 0)

Sure, you could use a Pydantic model, and get attribute access that way:

In [4]:
from pydantic import BaseModel

In [6]:
class Coordinate(BaseModel):
    x: int
    y: int
    
class Circle(BaseModel):
    center: Coordinate
    radius: int

class Rectangle(BaseModel):
    top_left: Coordinate
    bottom_right: Coordinate
    
class Data(BaseModel):
    circle: Circle
    rectangle: Rectangle

In [8]:
data = Data.model_validate_json(json_data)
data

Data(circle=Circle(center=Coordinate(x=0, y=0), radius=1), rectangle=Rectangle(top_left=Coordinate(x=-50, y=50), bottom_right=Coordinate(x=50, y=-50)))

In [9]:
center = data.circle.center.x, data.circle.center.y
center

(0, 0)

But you saw how much code we hade to write with Pydantic - and if all we wanted ewas easy attribute access to the elements of the JSON data, then Pydantic is probably overkill.

I showed you how `benedict` can help us in this regard - but let's say you don't like that library, or simply do not need the extra functionality it provides (which to my mind is the main benefit of `benedict`, not attribute access of dictionaries), then you can use Python `SimpleNamespace` class when you deserialize your JSON, not into a dictionary, but rather an instance of `SimpleNamespace`.

In [10]:
from types import SimpleNamespace

In [12]:
data = json.loads(
    json_data,
    object_hook=lambda x: SimpleNamespace(**x),
)

And now, we can get easy access to the JSON data elements using attribute notation:

In [13]:
center = data.circle.center.x, data.circle.center.y
center

(0, 0)

How easy was that!

Now, one snag is if the keys in the JSON objects themselves contain periods (`.`). That's perfectly valid in JSON, but will "break" our access.

All is not lost however:

In [14]:
json_data = '''{
    "a.1": {"x": 1, "y": 2},
    "a.2": {"x": 3, "y": 4}
}'''

In [15]:
data = json.loads(
    json_data,
    object_hook=lambda x: SimpleNamespace(**x),
)

In [16]:
data

namespace(a.1=namespace(x=1, y=2), a.2=namespace(x=3, y=4))

Now, we can use `getattr()` to get the attributes that we would not be able to access using simple dot notation:

In [17]:
getattr(data, "a.1").x

1

Even works for keys that contain spaces:

In [18]:
json_data = '''{
    "a 1": {"x": 1, "y": 2},
    "a 2": {"x": 3, "y": 4}
}'''

In [19]:
data = json.loads(
    json_data,
    object_hook=lambda x: SimpleNamespace(**x),
)

In [20]:
data

namespace(a 1=namespace(x=1, y=2), a 2=namespace(x=3, y=4))

In [21]:
getattr(data, "a 1").x

1

And there you go, a very simple, and pragmatic way to clearer code when dealing with JSON data.